# Вариант 0.1

In [29]:
import pandas as pd
from weather_parser import parser

In [30]:
DEP_NAME = "dep_datetime"
ARR_NAME = "arr_datetime"
INDEX_WEATHER_NAME = "date"
CITY_NAME = "center"
features_mask = ["wind_dir", "wind_speed", "Т(С)", "f(%)", "R(мм)", "R24(мм)"]

In [31]:
# Считываю датасет
dataset = pd.read_excel("../datasets/data.xlsx", index_col=[0])
# Сортирую по дате вылета, чтобы гаарнитровать что последний элемент укажет на самую последнюю дату
dataset = dataset.sort_values(DEP_NAME)
# Получаю список городов в датасете
uniq_centers = dataset["center"].unique()
# Добавляю в датасет новые фичи
for new_feature in features_mask:
    dataset[new_feature] = None

In [32]:
# Функция конверитрует дату и время в отдельные наборы
def get_data_info(idx, col):
    datatime = str(dataset.loc[idx, col])
    date, time = datatime.split()
    return (map(int, date.split('-')), map(int, time.split(":")))

In [33]:
def add_weather_features(idx, weather_data):
    # Получаю час взлета и посадки конкретного дрона 
    hour_dep, _, _ = get_data_info(idx, DEP_NAME)[1]
    hour_arr, _, _ = get_data_info(idx, ARR_NAME)[1]
    # Получаю год, месяц, день конкретного дрона
    y_dep, d_dep, m_dep = get_data_info(idx, DEP_NAME)[0]
    y_arr, d_arr, m_arr = get_data_info(idx, ARR_NAME)[0]
    # функция для доабвлению 0, чтоыб числа от 1-9 записывались как 01 и т.д
    add_zero = lambda num: f"0{num}" if len(str(num)) == 1 else num
    # Формирую диапозон который возьму ид дате о погоде 
    # при этом, так как значения идут сторого с шагом 3, все неподходящие я преобразую так: hour_dep // 3) * 3
    weather_data_idx_dep = f"{y_dep}.{add_zero(d_dep)}.{add_zero(m_dep)} {add_zero((hour_dep // 3) * 3)}"
    weather_data_idx_arr = f"{y_arr}.{add_zero(d_arr)}.{add_zero(m_arr)} {add_zero((hour_arr // 3) * 3)}"
    try:
        # Получаю нужные данные на добавление
        weather_data_for_item = weather_data.loc[weather_data_idx_dep:weather_data_idx_arr] 
        # Не все типы одинаковые, потому проходусь циклом по всем фичам и применяю либо моду для категориальных, лиюо среднее для числовых
        weather_info = weather_data_for_item.dtypes
        for col_name, dtype in zip(weather_info.index, weather_info):
            if dtype == "object":
                dataset.loc[idx, col_name] = weather_data_for_item[col_name].mode().iloc[0]
            else: 
                dataset.loc[idx, col_name] = weather_data_for_item[col_name].mean()
    except:
        print("Error!")

In [34]:
# Обрабатываю каждый уникальный город
for center in uniq_centers:
    # Получаю только записи нужного города, в которых указаны время взлёта и посадки
    dataset_center_idx = dataset[
        (dataset[CITY_NAME] == center) & 
        (dataset[DEP_NAME].notna() & dataset[ARR_NAME].notna())
    ].index.to_list()
    # Получаю самую первую дату
    y_f, m_f, _ = get_data_info(
        dataset_center_idx[0], 
        DEP_NAME
    )[0]
    # Получаю самую последнюю
    y_l, m_l, _ = get_data_info(
        dataset_center_idx[-1], 
        DEP_NAME
    )[0]
    # Теперь вытаскиваю данные за все нудные года и месяцы, идя от _f до _l
    weather_data = pd.concat(
        [
            parser(year, range(1, m_l + 1) if year == y_l else range(1, 12 + 1), center)
            for year in range(y_f, y_l + 1)
        ]
    ).set_index(INDEX_WEATHER_NAME)
    # Избавляюсь от multiindex
    weather_data.columns = [col[0] for col in weather_data.columns]
    weather_data.index = [idx[0] for idx in weather_data.index]
    # Все фичи кроме первой (которая категориальная) превращаю в числовые
    for feature in features_mask[1:]:
        weather_data[feature] = pd.to_numeric(weather_data[feature], errors="coerce")
    # для каждой записи нахожу... или нет, нкжные данные
    for idx in dataset_center_idx:
        add_weather_features(idx, weather_data[features_mask])

    print(f"{center} modified")

Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))

In [40]:
dataset.to_excel("../datasets/data_weather.xlsx")